# Example evaluation with rotating point sources

Requirements
- "Microphone Array Benchmark b11: Rotating Point Sources" dataset (https://doi.org/10.14279/depositonce-8460), files in subfolder "./td/"
- Acoular "rotating2024" branch (https://github.com/gherold/acoular/tree/rotating2024, last tested: 2025-03-24)
- Arra geometry file "ring64circ.xml"

Imports:

In [ ]:
import acoular as ac
ac.config.global_caching='none'
ac.config.h5library = "h5py"
#ac.config.h5library = "tables"
import pylab as plt
c0 = 343.

# select whether to rotate in modal or spatial domain
vra = 'mode-time domain'
#vra = 'space-time domain'


array geometry and focus grid:

In [ ]:
m = ac.MicGeomCirc(file = './ring64circ.xml')
channels = m.ringlist[0].mics

g = ac.RectGrid(x_min = -0.3,
                x_max =  0.3, 
                y_min = -0.3,
                y_max =  0.3,
                z = 0.5,
                increment = 0.01)

print(f'number mics: {m.num_mics:.0f}, number grid points: {g.size}')

plot setup

In [ ]:
plt.figure(1,(7,7))
plt.plot(0,0,'+')

for i in plt.arange(m.num_mics):
    plt.plot(m.pos[0,i],m.pos[1,i],'oC0')
    plt.text(m.pos[0,i]+0.005,m.pos[1,i]+0.005,str(i+1), fontsize=6)
    
plt.plot(g.pos[0],g.pos[1], '+k', ms=3)
plt.title(m.file)
plt.axis('equal')
plt.show()

Zeitsignal:

In [ ]:
# subcase
bc = 'b'

timedata = ac.aiaa.TimeSamplesAIAABenchmark(file = 'td/b11%sTimeSeries.h5' % bc)
print(f'signal duration: {timedata.num_samples/timedata.sample_freq} s')

trigger signal:

In [ ]:

timedata_trigger = ac.aiaa.TriggerAIAABenchmark(file='td/b11%sTimeSeriesOpt.h5' % bc)

plt.figure(2)

plt.plot(timedata_trigger.data[:int(1e4)])

trig_data = ac.Trigger(source=timedata_trigger, threshold =2)

Setup virtual rotation

In [ ]:
# New angle calculation
frot_data = ac.TrajectoryAnglesFromTrigger(trigger = trig_data,
                                        rot_direction = -1)

frot= frot_data.rps_average

env = ac.AxialRotatingFlowEnvironment(rotational_speed = -frot, 
                                      c=c0)

print(frot)

Setup virtual rotation

In [ ]:

if vra == 'mode-time domain': 
    # VRA (M-T)
    # space domain -> mode domain
    pmt = ac.SpaceModesTransformer(source = timedata, channel_order = channels)
    
    pmt_r = ac.VirtualRotatorModal(source = pmt, 
                                   rotational_speed = frot_data, 
                                   delay = plt.abs(g.z)/c0)
    
    # mode domain -> space domain
    p_rot = ac.SpaceModesTransformer(source = pmt_r, 
                                      channel_order = channels, 
                                      inverse=True)
elif vra == 'space-time domain':
    # VRA S-T
    p_rot = ac.VirtualRotatorSpatial(source = timedata, 
                                     channel_order = channels, 
                                     rotational_speed = frot_data, 
                                     delay = plt.abs(g.z)/c0)
else: raise NotImplementedError(f'Unknown rotation type "{vra}"')

Setup beamformer:

In [ ]:

# Sound propagation model
st = ac.SteeringVector(grid=g, mics=m, env=env)
# CSM
f = ac.PowerSpectraDR(
                    source = p_rot, 
                    window = 'Hanning', 
                    overlap = '50%',
                    block_size = 256)



# Beamformer
bf = ac.BeamformerBase(freq_data = f,
                       steer= st,
                       r_diag = True)



bfc = ac.BeamformerCleansc(freq_data = f,
                       steer= st,
                       r_diag = True)


bfcmf = ac.BeamformerCMF(freq_data = f,
                         steer= st,
                         r_diag = True,
                         method = 'NNLS') 

Calculation and visualization

In [ ]:

#algorithm
beamformer = bf
#freqbands
freqs = [2500, 5000, 10000] 
#bandsize
band = 3

nfreqs = len(freqs)

fig = plt.figure(3,figsize=(nfreqs*4, 4))  
nsub = 0

for freq in freqs:
    nsub+=1

    bbmap = beamformer.synthetic(freq,band)
    Lp = ac.L_p(bbmap)
    mx = Lp.max()
    

    plt.subplot(1,nfreqs, nsub) 

    #bf result
    plt.imshow(Lp.T,
               vmin = mx-20,
               vmax = mx,
               origin='lower',
               extent=g.extend(),
               interpolation='nearest',
               cmap=plt.cm.hot_r)


    plt.title(f'{freq} Hz, {vra}')


plt.show()